In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, fixed, Dropdown
import ipywidgets as widgets

import seaborn as sns
sns.set_context("poster")
sns.set_style('darkgrid')

The source file `WA_Residential_Wind_Exposure_201607_TCRM.CSV` can be found in HPRM D2017-84003. Download a local version (by using the 'Supercopy' option when right-clicking on the record), and change the path to the appropriate folder.

In [ ]:
inputFile = "C:\WorkSpace\data\derived\exposure\WA\WA_Residential_Wind_Exposure_201607_TCRM.CSV"
df = pd.read_csv(inputFile)

In [ ]:
localities = sorted(list(pd.unique(df['UCL_NAME_2011'])))
locdropdown = Dropdown(options=localities, description="Locality")
ages = sorted(list(pd.unique(df['YEAR_BUILT'])))

In [ ]:
def plotAgeDist(df, locality):
    fig, ax = plt.subplots(1, 2, figsize=(16,8))
    locdf = df[df['UCL_NAME_2011'] == locality]
    sns.countplot(x="YEAR_BUILT", data=locdf, order=ages, ax=ax[0],
                 palette=sns.color_palette("Set2", 8))

    ax[0].set_xlabel("Year built")
    ax[0].set_ylabel("Number")
    #locs0, labels0 = fig.xticks()
    plt.setp(ax[0].get_xticklabels(), rotation=90)
    ax[0].set_title("{0} - {1} residential buildings".format(locality, len(locdf.index)))
    
    suburblist = locdf[locdf['SUBURB_2015'].notnull()]['SUBURB_2015']
    suburbs = sorted(list(pd.unique(suburblist)))
    sns.countplot(x='SUBURB_2015', hue='YEAR_BUILT', data=locdf, order=suburbs, hue_order=ages,
                  palette=sns.color_palette("Set2", 8),ax=ax[1])
    ax[1].set_xlabel("Suburb")
    ax[1].set_ylabel("Number")
    labels = [item.get_text() for item in ax[1].get_xticklabels()]
    
    labels = [l.replace(' ', '\n') for l in labels]

    ax[1].set_xticklabels(labels)
    locs, labels = plt.xticks()

    plt.setp(labels, rotation=90)
    l = ax[1].legend(title="Year built", ncol=2)
    fig.tight_layout()
    plt.show()

In [ ]:
interact(plotAgeDist, df=fixed(df), locality=locdropdown)

There's two aspects to the age distribution - communities where there has been substantial growth since the last significant cyclone, and communities with a large proportion of older (pre-1980) era construction. 

TODO: 
1. Add a chart that ranks the localities by proportion of a selected age group. The list of age groups is already compiled (`ages`), just need to do the calculations to get proportions for the specified age group.
2. Add another figure that plots the predominant age group for each suburb in the locality. If there's a spatial layer of the boundaries for `SUBURB_2015`, then one could plot up a categorised map of the suburbs based on predominant age group.

In [ ]:
def plotBySuburb(df, locality):
    fig, ax = plt.subplots(1, 1, figsize=(16,12))
    locdf = df[df['UCL_NAME_2011'] == locality]
    suburblist = locdf[locdf['SUBURB_2015'].notnull()]['SUBURB_2015']
    suburbs = sorted(list(pd.unique(suburblist)))
    sns.countplot(x='SUBURB_2015', hue='YEAR_BUILT', data=locdf, order=suburbs, hue_order=ages,
                  palette=sns.color_palette("Set2", 8),ax=ax)
    ax.set_xlabel("Suburb")
    ax.set_ylabel("Number")
    locs, labels = plt.xticks()
    plt.setp(labels, rotation=90)
    l = ax.legend(title="Year built", ncol=2)
    plt.show()

In [ ]:
locdropdown2 = Dropdown(options=localities, description="Locality")
interact(plotBySuburb, df=fixed(df), locality=locdropdown2)